In [4]:
!pip install -q groq gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 5.0 MB/s eta 0:00:00


In [5]:
API_KEY = "ISI_API_KEY_DI_SINI"

import os
from groq import Groq

os.environ["GROQ_API_KEY"] = API_KEY
client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

In [10]:
SYSTEM_PROMPT = (
    "You are AIDA (AI Data Assistant), a formal and supportive mentor. "
    "Explain AI/Data Science topics clearly. If user asks for a short answer, be concise. "
    "If user wants in-depth explanation, provide structured, detailed responses with examples."
)

def detect_mode(text):
    t = text.lower()
    if any(k in t for k in ["singkat", "ringkas", "brief", "summary"]):
        return "short"
    if any(k in t for k in ["detail", "lebih dalam", "lengkap", "detailed"]):
        return "detailed"
    return "default"

def build_messages(history, user, mode):
    msgs = [{"role": "system", "content": SYSTEM_PROMPT}]
    for role, text in history:
        msgs.append({"role": role, "content": text})
    final_user = f"Jelaskan secara singkat: {user}" if mode=="short" else (
                 f"Jelaskan secara mendalam: {user}" if mode=="detailed" else user)
    msgs.append({"role": "user", "content": final_user})
    return msgs

def get_reply(messages):
    res = client.chat.completions.create(
        messages=messages,
        model="llama-3.3-70b-versatile",
        max_tokens=512,
        temperature=0.1,
    )
    return res.choices[0].message.content


In [13]:
import gradio as gr

custom_css = """
@import url('https://fonts.googleapis.com/css2?family=Nunito:wght@400;600&display=swap');

* { font-family: 'Nunito', sans-serif !important; }
.gradio-container {background-color: #F3E8FF !important;}
.user-message {background-color: #E9D5FF !important; color: #000 !important;}
.bot-message {background-color: #FFFFFF !important; color: #000 !important;}
button {background-color: #A78BFA !important; color: white !important;
        border-radius: 999px !important; padding: 6px 14px !important;}
"""

def respond(user_message, history):
    history = history or []

    mode = detect_mode(user_message)
    messages = build_messages([(m["role"], m["content"]) for m in history], user_message, mode)

    try:
        reply = get_reply(messages)
    except Exception as e:
        reply = f"[Error: {e}]"

    history.append({"role": "user", "content": user_message})
    history.append({"role": "assistant", "content": reply})
    return history, ""

with gr.Blocks(css=custom_css) as demo:
    gr.Markdown("### AIDA — AI Data Science Assistant\n*Created by **Anindia** for Hacktiv8 Final Project*")
    chatbot = gr.Chatbot(height=500, type="messages")
    with gr.Row():
        inp = gr.Textbox(placeholder="Tanya seputar AI/Data Science...", show_label=False)
        btn = gr.Button("Kirim")
    btn.click(respond, [inp, chatbot], [chatbot, inp])
    inp.submit(respond, [inp, chatbot], [chatbot, inp])

demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1e93e8538f46384d07.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
